<a href="https://colab.research.google.com/github/ChaejinE/MyPytorch/blob/main/d_NN_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview
- NN은 데이터셋에 대한 연산 수행하는 layer/module로 구성
- torch.nn namespace : 신경망을 구성하는 데 필요한 모든 구성요소 제공
- PyTorch의 모든 모듈은 nn.Module의 subclass이다.
- NN은 다른 모듈(layer)로 구성된 모듈이다.

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


# Class Definition
- NN 모델을 nn.Module의 subclass로 정의
- __init__ : 신경망 layer 들을 초기화
- nn.Module 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산을 구현한다.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
# NN Instance 생성 및 device로 이동한 뒤 구조를 출력
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class : {y_pred}")

Predicted class : tensor([6], device='cuda:0')


# Layer

In [ ]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


- nn.Flatten
  - 28x28 -> 784 값을 갖는 연속된 배열로 변환한다.
  - dim = 0의 미니배치 차원은 유지된다.

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


- nn.Linear
  - 저장된 wiehgt, bias를 이용해 linear transformation을 적용한다.

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


- nn.ReLU
  - 비선형 activation : input과 output 사이에 복잡한 관계를 만든다.
  - linear transform 후 적용되어 nonlinearity를 도입해 신경망이 다양한 현상을 학습할 수 있도록 돕는다.

In [ ]:
print(f"Before ReLU : {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU : {hidden1}\n\n")

Before ReLU : tensor([[-0.2829,  0.3555, -0.0149,  0.3796,  0.0906, -0.1055,  0.0634, -0.1396,
          0.3259,  0.2247,  0.2648, -0.2033,  0.4946,  0.4724, -0.3663,  0.3642,
          0.1579, -0.2760, -0.1618, -0.1720],
        [-0.0419,  0.3077,  0.4955,  0.2757, -0.0429,  0.1146, -0.2720, -0.1575,
          0.5855,  0.2073,  0.1545, -0.5847,  0.6437,  0.0528, -0.0662,  0.5041,
          0.0113,  0.2521, -0.0204,  0.2070],
        [-0.2373,  0.2796, -0.0744,  0.2232, -0.3046,  0.0943,  0.0015,  0.2434,
          0.0755,  0.3028,  0.3247,  0.0397,  0.3525,  0.2407, -0.1122,  0.2359,
         -0.0410, -0.1049, -0.0466,  0.0135]], grad_fn=<AddmmBackward>)


After ReLU : tensor([[0.0000, 0.3555, 0.0000, 0.3796, 0.0906, 0.0000, 0.0634, 0.0000, 0.3259,
         0.2247, 0.2648, 0.0000, 0.4946, 0.4724, 0.0000, 0.3642, 0.1579, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.3077, 0.4955, 0.2757, 0.0000, 0.1146, 0.0000, 0.0000, 0.5855,
         0.2073, 0.1545, 0.0000, 0.6437, 0.0528, 0.0

- nn.Sequential
  - 순서를 갖는 모듈의 컨테이너
  - 순차적으로 정의된 모듈로 전달된다.

In [16]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
logits

tensor([[ 0.4686, -0.3211,  0.2432, -0.1934, -0.0009,  0.0764,  0.2137,  0.2884,
         -0.2995, -0.2634],
        [ 0.3122, -0.2760,  0.2729, -0.0987,  0.0462,  0.1022,  0.0392,  0.2269,
         -0.1556, -0.2386],
        [ 0.2391, -0.1591,  0.3571, -0.1791, -0.0063,  0.1083,  0.1795,  0.1461,
         -0.3258, -0.2167]], grad_fn=<AddmmBackward>)

In [19]:
logits.size()

torch.Size([3, 10])

- nn.Softmax()
  - dim : 값의 합이 1이 되는 차원을 나타낸다
  - 모델의 각 class에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 scale 된다.

In [49]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [50]:
pred_probab

tensor([[0.1510, 0.0685, 0.1205, 0.0779, 0.0944, 0.1020, 0.1170, 0.1261, 0.0700,
         0.0726],
        [0.1309, 0.0727, 0.1259, 0.0868, 0.1003, 0.1061, 0.0996, 0.1202, 0.0820,
         0.0755],
        [0.1224, 0.0822, 0.1377, 0.0806, 0.0958, 0.1074, 0.1153, 0.1115, 0.0696,
         0.0776]], grad_fn=<SoftmaxBackward>)

- 모델 매개변수
  - 신경망 내부 layer들은 parameterize된다.
  - 학습 중 최적화 되는 weight, bias와 연관지어 진다.
  - nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 track(추적)된다.
  - parameters() or named_parameters()로 모든 매개변수에 접근할 수 있게된다.

In [51]:
print("Model structure : ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer : {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure :  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer : linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0103, -0.0101,  0.0141,  ..., -0.0073,  0.0335, -0.0314],
        [-0.0200, -0.0308, -0.0157,  ..., -0.0038, -0.0243, -0.0218]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer : linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0115, 0.0301], device='cuda:0', grad_fn=<SliceBackward>) 

Layer : linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0434, -0.0056, -0.0127,  ..., -0.0386, -0.0200, -0.0245],
        [-0.0241, -0.0035,  0.0162,  ..., -0.0051, -0.0290,  0.0086]],
       device='cu